# Business Project

Introduction:

New York is the most populous city in the United States. Also, the heart of the global financial market and This project will analyze which Hotel would be appropriate for the tourist to have nightlife on the trip to New York. For those who want to have a nice experience of hostel service after their nightlife, the content of this report is also suitable. This project will analyze the location of the convenience store, bar, and hostel.

Data section:

The report will use the data from the Foursquare API.the data will include the location of the hotel, bar, and convenience store in NewYork City. Then, The hostel, bar, and convenience store will be mapped by the folium library. The map visualizes the location of the venues above that will demonstrate the distribution of the bar and hotel. The hostel close to the bar and the convenience store would be selected. The selected group of hotels would be compared by the rating from Foursquare. Finally, the recommodation will be made by these process.

Methodology section:

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
address = 'New York'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [3]:
CLIENT_ID = '5G34KH1WV3TBA5XW2PTG0WORN3NE5WETRKTE1MFO41XCVSTZ' # your Foursquare ID
CLIENT_SECRET = '2STZQANK0KBEMBEFBJNUUMRTVL3DYGTJ4H3ZWEWP41PQCSHS' # your Foursquare Secret
VERSION = '20200522' # Foursquare API version
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5G34KH1WV3TBA5XW2PTG0WORN3NE5WETRKTE1MFO41XCVSTZ
CLIENT_SECRET:2STZQANK0KBEMBEFBJNUUMRTVL3DYGTJ4H3ZWEWP41PQCSHS


In [4]:
search_query = ''
categoryId='4bf58dd8d48988d116941735' #id for bar

radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT, categoryId)

search_query1 = ''
categoryId1='4bf58dd8d48988d1fa931735' # for hostel
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query1, radius, LIMIT, categoryId1)

search_query2 = ''
categoryId2='4d954b0ea243a5684a65b473' # for Convenience Store
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius, LIMIT, categoryId2)


In [5]:
results = requests.get(url).json()
results1 = requests.get(url1).json()
results2 = requests.get(url2).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues1 = results1['response']['venues']
venues2 = results2['response']['venues']


# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe1 = json_normalize(venues1)
dataframe2 = json_normalize(venues2)


dataframe.shape,
dataframe1.shape,
dataframe2.shape

(25, 18)

In [6]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]



# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]






In [7]:
dataframe_filtered1

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,"The Beekman, A Thompson Hotel",Hotel,123 Nassau St,US,New York,United States,Beekman St,182,"[123 Nassau St (Beekman St), New York, NY 1003...","[{'label': 'display', 'lat': 40.7111725, 'lng'...",40.711173,-74.006702,NaN,10038,NY,56d8c0f8498edb854f926e6a
1,Aloft Manhattan Downtown - Financial District,Hotel,49-53 Ann St,US,New York,United States,NaN,266,"[49-53 Ann St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.71039382975258...",40.710394,-74.006704,NaN,10038,NY,547dc381498ec37f84f01439
2,Moxy NYC Downtown,Hotel,26 Ann St,US,New York,United States,NaN,283,"[26 Ann St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.71066152473766...",40.710662,-74.007969,NaN,10038,NY,5bd4cc646adbf5002c1ced5c
3,Artezen Hotel,Hotel,24 John St,US,New York,United States,NaN,420,"[24 John St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.709585, 'lng':...",40.709585,-74.008780,NaN,10038,NY,5a91fa58e679bc57ca94db72
4,Hampton Inn by Hilton,Hotel,320 Pearl St,US,New York,United States,NaN,544,"[320 Pearl St, New York, NY 10038, United States]","[{'label': 'display', 'lat': 40.70889232606523...",40.708892,-74.002002,NaN,10038,NY,4a8ed915f964a520e11220e3
5,Duane Street Hotel,Hotel,130 Duane St,US,New York,United States,Church St,368,"[130 Duane St (Church St), New York, NY 10013,...","[{'label': 'display', 'lat': 40.7158584594727,...",40.715858,-74.007446,NaN,10013,NY,4ae77379f964a5206cab21e3
6,Holiday Inn New York City - Wall Street,Hotel,51 Nassau St,US,New York,United States,btwn Maiden Ln & Liberty St,493,"[51 Nassau St (btwn Maiden Ln & Liberty St), N...","[{'label': 'display', 'lat': 40.70897670394747...",40.708977,-74.009123,NaN,10038,NY,4b0017f7f964a520bb3a22e3
7,Millennium Hilton,Hotel,55 Church St,US,New York,United States,at Fulton St,406,"[55 Church St (at Fulton St), New York, NY 100...","[{'label': 'display', 'lat': 40.711114, 'lng':...",40.711114,-74.010333,Financial District,10007,NY,510ad6e2e4b0a2ecec03b322
8,Aka Tribeca,Hotel,85 W Broadway,US,New York,United States,Chambers,388,"[85 W Broadway (Chambers), New York, NY 10007,...","[{'label': 'display', 'lat': 40.715285, 'lng':...",40.715285,-74.009151,NaN,10007,NY,5c982c89a22db7003842c21a
9,Four Seasons,Hotel,30 Park Place,US,New York,United States,NaN,213,"[30 Park Place, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71292701984652...",40.712927,-74.008530,NaN,10007,NY,57c06f2e498e1de2b349c99e


In [8]:
# create map of New York using latitude and longitude values
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15
                    ) 

# add markers to map
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map) 
    
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.name):folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map) 
    

for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.name):folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map) 
    
    
venues_map


Central Harlem 10026, 10027, 10030, 10037, 10039
Chelsea and Clinton 10001, 10011, 10018, 10019, 10020, 10036
East Harlem 10029, 10035
Gramercy Park and Murray Hill	10010, 10016, 10017, 10022
Greenwich Village and Soho	10012, 10013, 10014
Lower Manhattan	10004, 10005, 10006, 10007, 10038, 10280
Lower East Side	10002, 10003, 10009
Upper East Side	10021, 10028, 10044, 10065, 10075, 10128
Upper West Side	10023, 10024, 10025
Inwood and Washington Heights	10031, 10032, 10033, 10034, 10040

# Data cleaning

In [9]:
dataframe_filtered2.to_csv('cs.csv')
df2=pd.read_csv('cs.csv')
df2 = df2.filter(['name','postalCode','id'], axis=1)
df2.rename(columns={"postalCode":"district","name":"convenience store"},inplace=True)
df2.replace(to_replace = [10004, 10005, 10006, 10007, 10038, 10280] ,value ="Lower Manhattan", inplace = True) 
df2.replace(to_replace = [10012, 10013, 10014] ,value ="Greenwich Village and Soho", inplace = True) 
df2.replace(to_replace = [10278] ,value ="Tribeca", inplace = True) 
df2[['district']] = df2[['district']].fillna(value="Lower Manhattan")

dataframe_filtered.to_csv('cs.csv')
df=pd.read_csv('cs.csv')
df = df.filter(['name','postalCode','id'], axis=1)
df.rename(columns={"postalCode":"district","name":"bar"},inplace=True)
df.replace(to_replace = [10004, 10005, 10006, 10007, 10038, 10280] ,value ="Lower Manhattan", inplace = True) 
df.replace(to_replace = [10012, 10013, 10014] ,value ="Greenwich Village and Soho", inplace = True) 
df.replace(to_replace = [10002, 10003, 10009] ,value ="Lower East Side", inplace = True) 
df.replace(to_replace = [10278,10279,10271] ,value ="Tribeca", inplace = True) 
df[['district']] = df[['district']].fillna(value="Lower Manhattan")

dataframe_filtered1.to_csv('cs.csv')
df1=pd.read_csv('cs.csv')
df1 = df1.filter(['name','address','postalCode','id'], axis=1)
df1.rename(columns={"postalCode":"district","name":"hostel"},inplace=True)
df1.replace(to_replace = [10004, 10005, 10006, 10007, 10038, 10280] ,value ="Lower Manhattan", inplace = True) 
df1.replace(to_replace = [10012, 10013, 10014] ,value ="Greenwich Village and Soho", inplace = True) 
df1.replace(to_replace = [10002, 10003, 10009] ,value ="Lower East Side", inplace = True) 
df1.replace(to_replace = [10278] ,value ="Tribeca", inplace = True)
df1[['district']] = df1[['district']].fillna(value="Lower Manhattan")

In [10]:
df.head() #check the data 

,bar,district,id
0,DMK Burger Bar,Lower Manhattan,4d571f67c0432d43b313ffb1
1,The Bar Room at Temple Court,Lower Manhattan,57f0689d498e7d49d9189369
2,Manhattan Proper,Lower Manhattan,4fb7ef4b7b0cd3a686313d43
3,Five & Dime,Lower Manhattan,59c90da3d807ee5bc5ce4ac6
4,The Beekman Pub,Lower Manhattan,43695300f964a52088291fe3


In [11]:
df1.head()#check the data 

,hostel,address,district,id
0,"The Beekman, A Thompson Hotel",123 Nassau St,Lower Manhattan,56d8c0f8498edb854f926e6a
1,Aloft Manhattan Downtown - Financial District,49-53 Ann St,Lower Manhattan,547dc381498ec37f84f01439
2,Moxy NYC Downtown,26 Ann St,Lower Manhattan,5bd4cc646adbf5002c1ced5c
3,Artezen Hotel,24 John St,Lower Manhattan,5a91fa58e679bc57ca94db72
4,Hampton Inn by Hilton,320 Pearl St,Lower Manhattan,4a8ed915f964a520e11220e3


In [12]:
df2.head()#check the data 

,convenience store,district,id
0,Duane Reade,Lower Manhattan,4b22833df964a520174824e3
1,7-Eleven,Lower Manhattan,5012af2fe4b07c3cf2288456
2,Duane Reade,Lower Manhattan,4b252930f964a520266d24e3
3,7-Eleven,Lower Manhattan,4b3d4ee9f964a520189225e3
4,Duane Reade,Lower Manhattan,4b7ecb98f964a520130030e3


10	Fun City Times Square	215 Broadway	Lower Manhattan	 4c6bad15599fbe9a412b933f
24	The Frederick Hotel	95 W Broadway	Lower Manhattan	 59e391c3e1f22816d1b09cf5
25	The Frederick Hotel	95 West Broadway	Lower Manhattan	 4bed7bbebac3c9b60157fee9
22	Cosmopolitan Hotel - TriBeCa	95 W Broadway	Lower Manhattan	 4a3a9952f964a52001a01fe3
16	Smythe Thompson Hotel	86 W Broadway	Lower Manhattan	 52855bf511d21362ef6f329f
13	Aka Tribeca	85 W Broadway	Lower Manhattan	 5c982c89a22db7003842c21a
18	Smyth Hotel	85 W Broadway	Lower Manhattan	 49efcc88f964a52006691fe3
19	Residence Inn by Marriott New York Downtown Ma...	170 Broadway	Lower Manhattan	 54aed577498ee5fc196739f7


# Hostel in West Broadway

In [13]:
venue_id = '4c6bad15599fbe9a412b933f',  # ID of Fun City Times Square
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('Fun City Times Square has not been rated yet.')

    
    
#4ea2297f8231f6dc69ebf699 Red lipstick

venue_id = '4ea2297f8231f6dc69ebf699',  # ID of Red lipstick
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('Red lipstick has not been rated yet.')


venue_id = '59e391c3e1f22816d1b09cf5' # ID of The Frederick Hotel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('The Frederick Hotel:',result['response']['venue']['rating'])
except:
    print('The Frederick Hotel has not been rated yet.')


venue_id = '4a3a9952f964a52001a01fe3' # ID of Cosmopolitan Hotel - TriBeCa
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print('Cosmopolitan Hotel - TriBeCa:',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

venue_id ='52855bf511d21362ef6f329f' # ID of Smythe Thompson Hotel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('Smythe Thompson Hotel has not been rated yet.')

venue_id = '5c982c89a22db7003842c21a' # ID of Aka Tribeca
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('Aka Tribeca has not been rated yet.')
    
venue_id = ' 49efcc88f964a52006691fe3 ' # ID of Smyth Hotel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('Smyth Hotel has not been rated yet.')


Fun City Times Square has not been rated yet.
Red lipstick has not been rated yet.
The Frederick Hotel has not been rated yet.
This venue has not been rated yet.
Smythe Thompson Hotel has not been rated yet.
Aka Tribeca has not been rated yet.
Smyth Hotel has not been rated yet.


# Greenwich Village and Soho 

In [14]:
venue_id = '4ae77379f964a5206cab21e3' # ID of  Duane Street Hotel

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('Duane Street Hotel:',result['response']['venue']['rating'])
except:
    print('Duane Street Hotel has not been rated yet.')
    

venue_id = '56e20413498ec9e0d11d2e9d' # ID of  Duane Street Hotel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('Eight Beta Suite:',result['response']['venue']['rating'])
except:
    print('Eight Beta Suite has not been rated yet.')    
    


Duane Street Hotel has not been rated yet.
Eight Beta Suite has not been rated yet.


# Four Seasons Hotel New York Downtown

In [15]:
venue_id = '57c640ad498e74977f98372f' # ID of  Millennium Hilton 

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('Four Seasons Hotel :',result['response']['venue']['rating'])
except:
    print('Four Seasons Hotel has not been rated yet.')

Four Seasons Hotel has not been rated yet.


# Residence Inn by Residence Inn by Marriott New York Downtown Manhattan

In [16]:
venue_id = '54aed577498ee5fc196739f7' # ID of Residence Inn by Residence Inn by Marriott New York Downtown Manhattan
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('Residence Inn by Marriott New York Downtown Manhattan',result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


# Millennium Hilton 

In [17]:
venue_id = '510ad6e2e4b0a2ecec03b322' # ID of  Millennium Hilton 

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('Millennium Hilton :',result['response']['venue']['rating'])
except:
    print('Millennium Hilton has not been rated yet.')

Millennium Hilton has not been rated yet.


# Acquis Executive Lounge	

In [18]:
venue_id = '4e9481cc5c5c9bf8ad96ca03',  # ID of Acquis Executive Lounge	
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print(result['response']['venue']['rating'])
except:
    print('Acquis Executive Lounge	 has not been rated yet.')

Acquis Executive Lounge	 has not been rated yet.


# JC's Hotel New York and The Beekman, A Thompson Hotel

In [19]:
#26	JC's Hotel New York	NaN	Lower Manhattan	4bd1c46f5e0cce728cf5a184
#The Beekman, A Thompson Hotel	123 Nassau St	Lower Manhattan	56d8c0f8498edb854f926e6a

venue_id = '4bd1c46f5e0cce728cf5a184',  # ID of JC's Hotel New York
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('JCs Hotel New York:',result['response']['venue']['rating'])
except:
    print( 'JCs Hotel New York has not been rated yet.')
    

venue_id = '56d8c0f8498edb854f926e6a',  # ID of The Beekman, A Thompson Hotel
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

try:
    print('The Beekman, A Thompson Hotel:',result['response']['venue']['rating'])
except:
    print( 'The Beekman, A Thompson Hotel has not been rated yet.')

JCs Hotel New York has not been rated yet.
The Beekman, A Thompson Hotel has not been rated yet.


# Clustering:

In [20]:
df33 = pd.DataFrame({'Hotel':['Aka Tribeca','Smythe Thompson Hotel','Smyth Hotel','The Frederick Hotel','Cosmopolitan Hotel - TriBeCa','The Frederick Hotel','Red lipstick','Duane Street Hotel','Eight Beta Suite','Four Seasons Hotel New York Downtown','Millennium Hilton','Acquis Executive Lounge','JCs Hotel New York','The Beekman, A Thompson Hotel'],
                     'street':['West Broadway','West Broadway','West Broadway','West Broadway','West Broadway','West Broadway','West Broadway','Greenwich Village and Soho','Greenwich Village and Soho','Barclay ','Church St','Broadway','Nassau street','Nassau street']})
df33=df33.groupby('street').count()
df33.reset_index(inplace=True) 
df32 = pd.DataFrame({'Latitude':[40.7073138374,40.712842,40.7078,40.715858,40.7093,40.715285],  
                     'Longitude':[-74.0054499782,-74.007740,-74.0129,-74.007446,-74.0087,-74.009151]})  
df333 = pd.concat([df33,df32],axis=1, join_axes=[df32.index])
df333


,street,Hotel,Latitude,Longitude
0,Barclay,1,40.707314,-74.005450
1,Broadway,1,40.712842,-74.007740
2,Church St,1,40.707800,-74.012900
3,Greenwich Village and Soho,2,40.715858,-74.007446
4,Nassau street,2,40.709300,-74.008700
5,West Broadway,7,40.715285,-74.009151


In [28]:
kclusters = 3

df333c = df333.drop('street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df333c)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 0, 0, 1], dtype=int32)

In [29]:
df333c.insert(0, 'Cluster Labels', kmeans.labels_) #assigned clusters label

In [30]:
df333c #check the group

,Cluster Labels,Hotel,Latitude,Longitude
0,2,1,40.707314,-74.005450
1,2,1,40.712842,-74.007740
2,2,1,40.707800,-74.012900
3,0,2,40.715858,-74.007446
4,0,2,40.709300,-74.008700
5,1,7,40.715285,-74.009151


In [32]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df333c['Latitude'], df333c['Longitude'], df333['street'], df333c['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


Results section:

it can only return the rating of The Frederick Hotel,Residence Inn by Marriott and Cosmopolitan Hotel - TriBeCa.The Frederick Hotel and Residence Inn by Marriott both rated at 8. both of these hotels have the convenience stole nearby.

Discussion section:

Discussion:
From the result above, it can be observed that many selected hotels have not been rated yet. However, it can still make some progress. If you want to stay in West Broadway, The Frederick Hotel should be a good choice based on our limited information. For Greenwich Village and Soho, it seems that it is not a place to compare with others. If the customer doesn't want to live in a crowded bar area, Four Seasons Hotel and Millennium Hilton would be recommended. Also, It can be observed that in the particular area, The hotel is quite close to the bar. it may be due to the complement relationship between these 2 services.
Difficulty:
Data searching and clearing is time-consuming. My original idea is to find out the distribution of the Michelin restaurant in Hong Kong/Tokyo. However, there are not the categories of “Michelin” in the foursquare. The data from foursquare.com is not enough for doing this research, there are only a few ratings of the hotel that can be used:(




Conclusion:

All in All, this project is finish. if someone follow the suggestion of this report which i suppose no one will do that, please tell me about the experience. there are also some new ideas after doing this report. for example, is there the positive relation between the number of the hostel and the bar. does the hostel close to the bar will have higher attendance than the  hotel routine from the bar? i think it is the interesting question to investigate. i hope you would like this.
